In [ ]:
import duckdb

In [ ]:
con = duckdb.connect(database="dados_duckdb.db", read_only=False) #estabelecendo a conexão

In [ ]:
df = con.execute("SELECT * FROM bronze_z0019").fetchdf()
df.head(10) 

# recuperação dos dados que criamos     
# Para ver o resultado colocamos dentro de um data frame (df) e passamos a função --> fetchdf

In [ ]:
# Usando o SQL para pegar o último registro por ID - Como fazer isso?
# Primeiro -> Ordenar os registros com base na data_ingestao
# Vai indentificar qual é o último e fazemos a alteração.


In [ ]:
df = con.execute("""
                 SELECT *  
                 FROM (
                        SELECT *, ROW_NUMBER() OVER (PARTITION BY NATBR ORDER BY data_ingestao DESC) AS row
                        FROM bronze_z0019
                        WHERE data_ingestao >= '2025-01-11'  
                 ) WHERE row = 1
                 """).fetchdf()

df.head(10) 
# WHERE data_ingestao >= '2025-01-11' --> Filtra o dia (Quando data_ingestao for maior ou igual a 28 / 01 / 2026)
# Não teve alteração, pois os registros são do dia 
# adicionamos uma nova informação --> Número da linha, Partições NATBR que é a coluna, 


In [ ]:
## explicando de forma mais detalhada o código: 
# PARTITION BY NATBR  ---> 
# ORDER BY data_ingestao DESC
# ROW_NUMBER()

In [ ]:
df_final = df.drop(columns=['nome_arquivo', 'data_ingestao', 'row']) #apagando as colunas
df_final = df_final.rename(columns={"NATBR": "id"}) # Renomenado a coluna
df_final = df_final.rename(columns={"MAKTX": "nm_produto"})
df_final = df_final.rename(columns={"WERKS": "id_categoria"})
df_final = df_final.rename(columns={"MANIS": "id_fornecedor"})
df_final = df_final.rename(columns={"LABST": "vl_preco"})
df_final.head(10) 

In [ ]:
# fazendo a ingestão desses dados. 

df2 = df_final

df_final
# df2.head(10)

In [ ]:
df2 = df_final
df2 = df2.astype()

In [ ]:
df_final.dtypes  # olhando o formato --> são objetos, mas se quero fazer um cálculo?

In [ ]:
df2 = df_final

df2 = df2.astype(
    {
        'id': int,
        'nm_produto' : str,
        'id_categoria' : str,
        'id_fornecedor' : int,
        'vl_preco' : float
    }
)

df2.dtypes

In [ ]:
# para armazenar esses dados --> criamos as tabelas. 

con.execute("""CREATE TABLE IF NOT EXISTS produtos (
            id BIGINT,
            nm_produtos TEXT,
            id_categoria TEXT,
            id_fornecedor BIGINT,
            vl_preco FLOAT
            )
""")

In [ ]:
df2.head(10)

In [ ]:
con.execute("INSERT INTO produtos SELECT * from  df2 ") # inserindo no Data frame

In [ ]:
df_resultado = con.execute('select * from produtos').fetchdf()
df_resultado.head(10)

In [ ]:
con.close()